In [12]:
import os

In [ ]:

from dotenv import load_dotenv
load_dotenv()

In [15]:
! pip install langchain_huggingface

In [2]:
!pip install -q langchain-openai langchain-core requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 6.1 MB/s eta 0:00:00


In [42]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
import requests

In [43]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [44]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [6]:
multiply.name

'multiply'

In [7]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [8]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [ ]:
! pip install

In [9]:
# tool binding

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint
model = HuggingFaceEndpoint(
    repo_id="gpt2",
    task="text-generation"
)
llm=ChatHuggingFace(llm=model)

In [55]:
llm.invoke('hi')

StopIteration: 

In [32]:
llm_with_tools = llm.bind_tools([multiply])

In [49]:
llm_with_tools.invoke('Hi how are you')

StopIteration: 

In [50]:
query = HumanMessage('can you multiply 3 with 1000')

In [51]:
messages = [query]

In [52]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [53]:
result = llm_with_tools.invoke(messages)

StopIteration: 

In [ ]:
messages.append(result)

In [ ]:
messages

In [ ]:
tool_result = multiply.invoke(result.tool_calls[0])

In [ ]:
tool_result

In [ ]:
messages.append(tool_result)

In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages).content

In [ ]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [ ]:
convert.args

In [ ]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

In [ ]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

In [ ]:
# tool binding
llm = ChatOpenAI()

In [ ]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [ ]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [ ]:
messages

In [ ]:
ai_message = llm_with_tools.invoke(messages)

In [ ]:
messages.append(ai_message)

In [ ]:
ai_message.tool_calls

In [ ]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages).content

In [ ]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

In [ ]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})